In [1]:
%load_ext jupyter_black
import sys

sys.path.append("../")

In [2]:
from __future__ import annotations
from typing import *

import pandas as pd
import numpy as np
from numpy.typing import NDArray, ArrayLike
import matplotlib.pyplot as plt

from src.mesoformer.typing import Self, Concatenate, TypeVarTuple, Unpack, NestedSequence
from src.mesoformer.generic import Array
from src.mesoformer.utils import normalized_scale, sort_unique


N = bool | int | slice

arr = Array([[1, 2, 3]])  # type: Array[[N, N], int]
mask = arr.is_in([1, 2])
print(arr, mask, arr[mask], sep="\n")

Array:
[[1 2 3]]
Array:
[[ True  True False]]
Array:
[1 2]


In [3]:
from src.mesoformer.typing import TensorLike
import torch


def f(x: TensorLike[[int, int], int]) -> TensorLike[[int, int], float]:
    return x / x


print(
    f(np.array([[1, 2, 3]])),  # Ok
    f(torch.tensor([[1, 2, 3]])),  # Ok
)

[[1. 1. 1.]] tensor([[1., 1., 1.]])


In [4]:
P0 = 1013.25  # - mbar
P1 = 25.0  # - mbar
STP = sort_unique([P0, *range(1000, 25 - 1, -25), P1]).astype(np.float_)[::-1]
ERA5_GRID_RESOLUTION = 30.0  # km / px
RATE = ERA5_GRID_RESOLUTION / 2
URMA_GRID_RESOLUTION = 2.5  # km / px

MESOSCALE_BETA = 200.0  # km

In [5]:
from src.mesoformer.typing import Array, N


def arange_troposphere(
    start: int = 1000,
    stop: int = 25 - 1,
    step: int = -25,
    *,
    p0=P0,
    p1=P1,
) -> NDArray[np.float_]:
    return sort_unique([p0, *range(start, stop, step), p1])[::-1]


arange_troposphere()

array([1013.25, 1000.  ,  975.  ,  950.  ,  925.  ,  900.  ,  875.  ,
        850.  ,  825.  ,  800.  ,  775.  ,  750.  ,  725.  ,  700.  ,
        675.  ,  650.  ,  625.  ,  600.  ,  575.  ,  550.  ,  525.  ,
        500.  ,  475.  ,  450.  ,  425.  ,  400.  ,  375.  ,  350.  ,
        325.  ,  300.  ,  275.  ,  250.  ,  225.  ,  200.  ,  175.  ,
        150.  ,  125.  ,  100.  ,   75.  ,   50.  ,   25.  ])

In [6]:
# P0 = 1013.25  # - mbar
# P1 = 25.0  # - mbar
# STP = sort_unique([P0, *range(1000, 25 - 1, -25), P1]).astype(np.float_)[::-1]
# ERA5_GRID_RESOLUTION = 30.0  # km / px
# RATE = ERA5_GRID_RESOLUTION / 2
# URMA_GRID_RESOLUTION = 2.5  # km / px

# MESOSCALE_BETA = 200.0  # km


# class Troposphere(Array[[N], np.float64]):
#     @classmethod
#     def arange(cls, *, p0=P0, p1=P1) -> Troposphere:
#         return cls(sort_unique([p0, *range(1000, 25 - 1, -25), p1]).astype(np.float64)[::-1])

#     @property
#     def log_p(self) -> NDArray[np.float64]:
#         return np.log(self._data)

#     @property
#     def p0(self) -> np.float64:
#         return self._data[0]

#     @property
#     def p1(self) -> np.float64:
#         return self._data[-1]


# trop = Troposphere.arange()
# mask = trop.is_in([1000])
# print(
#     trop,
#     mask,
#     trop[mask],
#     trop.p0,
#     sep="\n",
# )

In [7]:
from src.mesoformer.utils import normalize

P0 = 1013.25  # - mbar
P1 = 25.0  # - mbar
STP = sort_unique([P0, *range(1000, 25 - 1, -25), P1]).astype(np.float_)[::-1]
ERA5_GRID_RESOLUTION = 30.0  # km / px
RATE = ERA5_GRID_RESOLUTION / 2
URMA_GRID_RESOLUTION = 2.5  # km / px

MESOSCALE_BETA = 200.0  # km


def arange_troposphere(
    start: int = 1000,
    stop: int = 25 - 1,
    step: int = -25,
    *,
    p0=P0,
    p1=P1,
) -> NDArray[np.float_]:
    return sort_unique([p0, *range(start, stop, step), p1])[::-1]


array2string = lambda x: np.array2string(
    x,
    max_line_width=72,
    precision=2,
    separator=" ",
    floatmode="fixed",
)


def array_repr(**kwargs: NDArray) -> str:
    return "\n".join(f"{k}:\n{array2string(v)}" for k, v in kwargs.items())


class Mesoscale:
    @staticmethod
    def _create_scale(x: NDArray[np.float_], rate: float) -> NDArray[np.float_]:
        x = normalize(np.log(x))
        x *= rate
        x += 1
        return x

    def __init__(
        self,
        dx: float = 200.0,
        dy: float | None = None,
        km_px: float = URMA_GRID_RESOLUTION,
        *,
        mbars: list[float],
        rate: float = 1.0,
        troposphere: NDArray[np.float_] | None = None,
    ) -> None:
        super().__init__()

        troposphere = troposphere if troposphere is not None else arange_troposphere()
        self._mbars = p = sort_unique(mbars)[::-1]
        self._mask = mask = np.isin(troposphere, p)
        self._scale = scale = self._create_scale(troposphere, rate=rate)[::-1][mask]
        self.dx, self.dy = scale[np.newaxis] * np.array([[dx], [dy or dx]])

    @property
    def mbars(self) -> NDArray[np.float_]:
        return self._mbars

    def to_pandas(self) -> pd.DataFrame:
        return pd.DataFrame(
            {
                "dx": self.dx,
                "dy": self.dy,
                "px": self.dx / URMA_GRID_RESOLUTION,
                "py": self.dy / URMA_GRID_RESOLUTION,
            },
            index=pd.Index(self.mbars, name="hPa"),
        ).sort_index()

    def __repr__(self) -> str:
        return array_repr(
            mbars=self.mbars,
            dx=self.dx,
            dy=self.dy,
            px=self.dx / URMA_GRID_RESOLUTION,
            py=self.dy / URMA_GRID_RESOLUTION,
            scale=self._scale,
        )


ms = Mesoscale(200, 175, mbars=[P0, 925.0, 850.0, 700.0, 500.0, 300.0], rate=RATE)
ms

mbars:
[1013.25  925.00  850.00  700.00  500.00  300.00]
dx:
[ 200.00 1504.23 1885.10 2338.60 2704.87 2956.21]
dy:
[ 175.00 1316.20 1649.47 2046.27 2366.76 2586.68]
px:
[  80.00  601.69  754.04  935.44 1081.95 1182.48]
py:
[  70.00  526.48  659.79  818.51  946.70 1034.67]
scale:
[ 1.00  7.52  9.43 11.69 13.52 14.78]

In [8]:
# import torch
# from collections.abc import Sequence
# from typing import Union, SupportsIndex, overload
# import torch

# _ShapeLike = Union[SupportsIndex, Sequence[SupportsIndex]]


# # @overload
# # def normalize(x: NDArray[np.number]) -> NDArray[np.float32]:
# #     ...


# # @overload
# # def normalize(x: torch.Tensor) -> torch.Tensor:
# #     ...


# # def normalize(x: torch.Tensor | NDArray[np.number], **kwargs) -> NDArray[np.float32] | torch.Tensor:
# #     return x - x.min(**kwargs) / (x.max(**kwargs) - x.min(**kwargs))  # type: ignore
# # # def mesoscale(stp=STP, rate: float = 1.0) -> NDArray[np.float_]:
# # #     return scale(np.log(stp)[::-1], rate=rate)


# # normalize(
# #     np.array([1, 3]),
# # ), normalize(
# #     torch.tensor([1, 3]),
# )